In [ ]:
import time
import requests
from bs4 import BeautifulSoup
from pydantic import BaseModel

from ollama import Client
from ollama import chat
from ollama import ChatResponse

In [ ]:
import smtplib
import schedule
import time
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
class AstroPhAnalyzer:

    def __init__(self, _model):
        self.model = _model
        self.results = []

    def parse(self, _user_prompt):

        class output_format(BaseModel):
            output: str

        system_prompt = """                                                                                                                                                                      
        You are an extragalactic cosmologist.                                                                                                                                                    
        From the given text identify if any of the following keywords or related keywords is present (note that all are extragalactic processes):                                                
        - Galaxy                                                                                                                                                                                 
        - Galaxy formation                                                                                                                                                                       
        - Galaxy evolution                                                                                                                                                                       
        - Galaxy mergers                                                                                                                                                                         
        - Green valley                                                                                                                                                                           
        - Galaxy color                                                                                                                                                                           
        - Spiral galaxies                                                                                                                                                                        
        - Elliptical galaxies                                                                                                                                                                    
        - AGN                                                                                                                                                                                    
        - Active Galactic Nucleus                                                                                                                                                                
        - Active Galactic Nuclei                                                                                                                                                                 
        - NGC                                                                                                                                                                                    
        - Morphology-density relation                                                                                                                                                            
        - galaxy environment                                                                                                                                                                     
        - Large scale structure of the Universe                                                                                                                                                  
        - cosmic web                                                                                                                                                                             
        - cosmic filaments                                                                                                                                                                       
        - cosmic walls                                                                                                                                                                           
        - voids                                                                                                                                                                                  
        - Galaxy spin acquisition                                                                                                                                                                
        - galaxy spin alignment                                                                                                                                                                  
        - Galaxy properties                                                                                                                                                                      
        - Gas accretion into galaxies                                                                                                                                                            
        - Star formation.                                                                                                                                                                        
        - N-boby simulations                                                                                                                                                                     
        - TNG                                                                                                                                                                                    
        - Illustris                                                                                                                                                                              
        - Eagle                                                                                                                                                                                  
        - Photometric redshift                                                                                                                                                                   
        If so, then return 'true', otherwise return 'false'.                                                                                                                                     
        However, if any of the following keywords are present in the title or abstract then return 'false' unless the keyword is explicilty related to one of the above keywords:                
        - Wolf-Rayet                                                                                                                                                                             
        - axion                                                                                                                                                                                  
        - magnetar                                                                                                                                                                               
        - gravitational wave                                                                                                                                                                     
        - general relativity                                                                                                                                                                     
        - planets                                                                                                                                                                                
        - binaries                                                                                                                                                                               
        - planetary nebula                                                                                                                                                                       
        - nebula                                                                                                                                                                                 
        - dust                                                                                                                                                                                   
        - Protostar                                                                                                                                                                              
        - pulsars                                                                                                                                                                                
        - neutron star                                                                                                                                                                           
        - white dwarfs                                                                                                                                                                           
        - detectors                                                                                                                                                                              
        - supernova                                                                                                                                                                              
        - gamma ray                                                                                                                                                                              
        - neutrino                                                                                                                                                                               
        In your response avoid any preable or comments. Just output ONE word: 'true' or 'false'.                                                                                                 
        """
        messages = [
                {"role": "system", "content": system_prompt },
                {"role": "user", "content": _user_prompt}
            ]
        response = chat(
        messages=messages,
        model=self.model,
        format=output_format.model_json_schema(),)
        
        out = output_format.model_validate_json(response.message.content)
        return out.output
    
    def fetch_astro_ph_abstracts(self):
        """Fetch abstracts, titles, authors, and arXiv IDs from astro-ph webpage"""
        ASTRO_PH_URL   = 'https://arxiv.org/list/astro-ph/new'
        try:
            response = requests.get(ASTRO_PH_URL)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            counter = 0
            papers = []
            for i,dt in enumerate(soup.find_all('dt')):

                #print(i, len(soup.find_all('dt')), counter)
                #if counter > 2: continue
                
                dd = dt.find_next_sibling('dd')
                if dd:
                    # Extract arXiv ID
                    arxiv_id = dt.find('a', {'title': 'Abstract'})
                    arxiv_id = arxiv_id.text.strip() if arxiv_id else "Unknown"
                    
                    # Extract title and authors
                    meta = dd.find('div', class_='meta')
                    if meta:
                        title = meta.find('div', class_='list-title')
                        title = title.text.replace('Title:', '').strip() if title else "Unknown"
                        
                        authors = meta.find('div', class_='list-authors')
                        authors = authors.text.replace('Authors:', '').strip() if authors else "Unknown"
                        
                        abstract = meta.find('p')
                        abstract = abstract.text.strip() if abstract else ""

                        abstract_parsed = self.parse(abstract)

                        #--- Add valid papers to reading list
                        papers.append({
                            'arxiv_id': arxiv_id,
                            'title': title,
                            'authors': authors,
                            'abstract': abstract,
                            'value': abstract_parsed
                        })

                        if abstract_parsed == 'true':
                            counter +=1
                            
            return papers
        except Exception as e:
            print(f"Error fetching papers: {e}")
            return []
 

In [ ]:
def construct_html(_response):
    html_start = """
        <!DOCTYPE html>
    <html>
    <head>
        <style>
            h1 { font-size: 16px; font-weight: bold; }
            h2 { font-size: 14px; font-weight: bold; }
            p1 { font-size: 16px; font-weight: normal; }
            p2 { font-size: 12px; font-weight: normal; }
        </style>
    </head>
    <body>
    """
    html_end = """
    </body>
    </html> 
    """
    body = """
    <h1>astro-ph daily digest</h1>
    """

    #--- Add valid papers
    for i, item in enumerate(_response):
        if item['value'] == 'true':

            authors = item['authors'].split(',')
            if len(authors) > 5:
                authors = ', '.join(authors[0:5]) + ' et al.'
            else:
                authors = item['authors']

            body += f"""                                                                                                                                                                         
                <h2>{item['title']}</h2>                                                                                                                                                         
                <p1>{authors}  https://arxiv.org/abs/{item['arxiv_id'].split(':')[1]}.</p1>                                                                                                      
                <hr>                                                                                                                                                                             
                """
            
    body += f"""
        <br>
        <h1>Other papers:</h1>
    """
    for i, item in enumerate(_response):
        if item['value'] == 'false':
            body += f"""
                <p2>{item['title']} https://arxiv.org/abs/{item['arxiv_id'].split(':')[1]}.</p2>
                <br>
                """
    
    return html_start + body + html_end

In [ ]:
def send_email(_body):
    """Sends an email with the results of the hello() function."""
    sender_email = "coyote.hill.cam@gmail.com"
    receiver_email = "maragon@astro.unam.mx"
    password = "ruii shle lamr kckp"  # Use an app password, not your actual password

    # Email content
    subject = "Daily astro-ph Digest"
    
    # Create email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(_body, 'html'))
    
    try:
        # Connect to Gmail SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print("Email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")


In [ ]:
def daily_digest():
    analyzer = AstroPhAnalyzer('llama3.2:3b')

    #--- Fetching and analyzing astro-ph
    result = analyzer.fetch_astro_ph_abstracts()
    
    send_email(construct_html(result[0:4]))

***
***

In [ ]:
# Schedule the job to run every day at 6 AM
schedule.every().day.at("05:00").do(daily_digest)

if __name__ == "__main__":
    while True:
        schedule.run_pending()
        time.sleep(60)  # Wait a minute before checking again                                                                                                                                    
